# Kaggle TPS 202109

## Optuna LightGBM Starter

### Check GPU

In [ ]:
# Google Colab
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Sep 18 04:24:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Check Memory

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


### PIP Installs

Uninstall the colab default lightgbm, since it is not set up for GPU

In [ ]:
!pip uninstall lightgbm


Found existing installation: lightgbm 2.2.3
Uninstalling lightgbm-2.2.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/lightgbm-2.2.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/lightgbm/*
Proceed (y/n)? y
  Successfully uninstalled lightgbm-2.2.3


For now, git clone the lightgbm from the repo.  (There is possibly another GPU friendly install option, but it hasn't been verified to work)

Note that this will take up space, so this is being done before mounting Google Drive



In [ ]:
! git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;    
# unfortunately I forgot where I got the above, so cannot give them adequate credit

Cloning into 'LightGBM'...
remote: Enumerating objects: 23494, done.
remote: Counting objects: 100% (1171/1171), done.
remote: Compressing objects: 100% (631/631), done.
remote: Total 23494 (delta 731), reused 847 (delta 519), pack-reused 22323
Receiving objects: 100% (23494/23494), 18.12 MiB | 23.97 MiB/s, done.
Resolving deltas: 100% (17124/17124), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21731, done.        
remote: Counting objects: 100% (3/3), done.        
remote: 

Other untried option

```!pip install lightgbm --install-option=--gpu```


In [ ]:
# Next two lines are for Google Colab GPU and working with AutoGluon
!pip uninstall -y mkl
!pip install --upgrade mxnet-cu100

!pip install optuna

Found existing installation: mkl 2019.0
Uninstalling mkl-2019.0:
  Successfully uninstalled mkl-2019.0
     |████████████████████████████████| 352.6 MB 5.6 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 302 kB 12.8 MB/s 
     |████████████████████████████████| 80 kB 11.1 MB/s 
     |████████████████████████████████| 208 kB 79.6 MB/s 
     |████████████████████████████████| 75 kB 4.9 MB/s 
     |████████████████████████████████| 144 kB 93.9 MB/s 
     |████████████████████████████████| 111 kB 98.5 MB/s 
     |████████████████████████████████| 49 kB 7.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=5776c43d764c1215150f047aec941d6e5de4d67fb32fee42c2c6a82bff328b18
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfull

## Imports

In [ ]:
import random
import os
from google.colab import output
import optuna


from lightgbm import LGBMClassifier
import lightgbm as lgb


import pandas as pd

from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import pickle
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import roc_auc_score

#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

import logging

## Google Drive Mount

* This is used to save models and output
* Note that for this Kaggle competition, the train.csv and test.csv are quite large and take a while to upload and store.

In [ ]:
working_dir = 'MyDrive/kaggle/202109'
drive_dir = '/content/drive/'
model_dir = 'models'
output_dir = 'output'
from google.colab import drive
drive.mount(drive_dir, force_remount=True)  # Do this once only

mounted_drive = f'{drive_dir}/{working_dir}' #''
os.chdir(mounted_drive)


Mounted at /content/drive/


In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/.kaggle"

If you want to directly download the kaggle files to your Google Colab environment

From https://www.analyticsvidhya.com/blog/2021/05/10-colab-tips-and-hacks-for-efficient-use-of-it/

https://github.com/Kaggle/kaggle-api

You can pull the datasets directly from Kaggle
1.  Go to your account page on kaggle.com and scroll to API to get your token
2.  Download the token 
3.  Copy it to Google Drive - most people put it in the .kaggle directory
4.  Set enviornment variable KAGGLE_CONFIG_DIR
5.  ```
! pip install kaggle
kaggle competitions download -c tabular-playground-series-sep-2021```
6.  You may need to add logic to move the files where you want them

In [ ]:
#!pip install kaggle
#!kaggle competitions download -c tabular-playground-series-sep-2021

In [ ]:
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

# Read csv files
df = pd.read_csv('./train.csv')
test_df=pd.read_csv('./test.csv')


## Logging

In [ ]:
# Right now this is not working as expected, so this is pending
#logging.basicConfig(filename='./artifacts/Kaggle_202109_lgbm.log',level=logging.INFO)

# Massaging Data

These are taken from Kaggle notebooks (see commented code)


1.   Memory optimization 
     * scans each column and based on the numeric range, sets the type to the one that requires the least memory
     * improves performance because the datasets are very large
2.   Imputation
     * uses sklearn's iterative imputer which approximates the missing values based on "nearby" values.
     * improves scores
     * note that it takes a while, so for this notebook, the massaged data is being saved locally
3.   additional columns are added to indicate the number of rows missing data and the missing feature columns

In [ ]:
RUN_MASSAGE = False

In [ ]:
# https://www.kaggle.com/lucamassaron/autogluon-for-tabular-playground-sep-2021
# Derived from the original script https://www.kaggle.com/gemartin/load-data-reduce-memory-usage 
# by Guillaume Martin

# This checks the range of values in each column and resets the type 

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
df = reduce_mem_usage(df)
test_df = reduce_mem_usage(test_df)

Mem. usage decreased to 435.76 Mb (50.3% reduction)
Mem. usage decreased to 224.01 Mb (50.0% reduction)


In [ ]:
if RUN_MASSAGE:
  X = df.drop(columns = ['id', 'claim'])
  test_X = test_df.drop(columns = ['id'])

In [ ]:
#https://www.kaggle.com/lucamassaron/autogluon-for-tabular-playground-sep-2021
# Reference: https://www.kaggle.com/hsuchialun/tps-lightgbm-kfold
if RUN_MASSAGE:
    feats = list(X.columns[:-1])

    X['n_row_missing'] = X[feats].isna().sum(axis=1)
    test_X['n_row_missing'] = test_X[feats].isna().sum(axis=1)

    X['row_std'] = X[feats].std(axis=1)
    test_X['row_std'] = test_X[feats].std(axis=1)

    feats += ['n_row_missing', 'row_std']


In [ ]:
# MULTIVARIATE ITERATIVE IMPUTATION

if RUN_MASSAGE:
    
    imp = IterativeImputer(max_iter=10, random_state=0)
    imp.fit(X[feats].sample(n=10_000))

    X[feats] = imp.transform(X[feats])
    test_X[feats] = imp.transform(test_X[feats])

In [ ]:
if RUN_MASSAGE:
    pd.DataFrame(X).to_csv('./reduced_mem_imputed.csv', index=False)
    pd.DataFrame(test_X).to_csv('./reduced_mem_imputesd_test.csv', index=False)

In [ ]:
X = pd.read_csv('./reduced_mem_imputed.csv')
test_X = pd.read_csv('./reduced_mem_imputesd_test.csv') # this is the Kaggle test dataset

In [ ]:
y = df[['claim']]
N_SPLITS = 5 # if you're using cross_score_validation
EARLY_STOPPING_ROUNDS = 400
SEED = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Optuna 
* study - this is the tuning done over several iterations of different hyperparameters
* trial - this is the individual evaluation/trial of selected hyperparameters
* objective function - defines the logic that Optuna will use for a single trial
* callback - allows actions after the objective function has returned the ranking metric to Optuna.  

At this point Optuna does not save the best model for use after the study.  Instead it will return the best parameters of the study.  However the study can take a long time, so a callback is used to save the best model (as of yet) to a file.

**Note that there should be a sample file called best_{model_type} which will be replaced by the best model of the study.  Right now the job will err out, if the model isn't there. ** (And yes, more code can be written to automatically create that file or check it's existence, but it hasn't been done yet)

Due to the large training and test datasets, the objective function has been modified to work with train_test_split

In [ ]:
def lgbm_objective(trial):
   
    # hyperparameter setting, trial.suggest_uniform will suggest uniform hyperparameter
    
    
    kfolds = KFold(n_splits=N_SPLITS, shuffle=True)
    
    params = {
        "metric": "AUC",
        "verbosity": 0,
        "device": "gpu",
        "boosting_type": trial.suggest_categorical('boosting_type', ['gbdt']), # , 'dart'
        "subsample": trial.suggest_uniform('subsample', 0.5, 0.6),
        "learning_rate": trial.suggest_float('eta', 0.005, 0.015),
        "num_iterations": trial.suggest_int('num_iterations', 1000, 40000),
        "subsample_freq": 1,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-7, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-7, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),        
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    print(f'STARTING Params: {params}')
    lgbm_model = LGBMClassifier(**params,random_state=42)
    
    d_train = lgb.Dataset(X, label=y)

    #https://www.kaggle.com/aharless/trying-to-do-k-fold-cv-on-lightgbm
    cv_results = lgb.cv(params, d_train, early_stopping_rounds=EARLY_STOPPING_ROUNDS, nfold=N_SPLITS, metrics = 'auc', seed=42)
    
    #print(cv_results)
    #results['auc-mean'][-1]
    auc = cv_results['auc-mean']
    print('Trial LGBM AUC',np.amax(auc), np.amin(auc), np.sqrt(abs(np.mean(auc))))
    filename = f'./models/lgbm_{trial.number}'
    
    pickle.dump(lgbm_model, open(filename, 'wb'))
    
    return auc[-1] 


def lgbm_callback(study, trial):
  # from os import remove
    filename = f'./models/lgbm_{trial.number}'
    
    best_model_name = './models/best_lgbm'
    if study.best_trial.number == trial.number:
        os.remove(best_model_name)
        os.rename(filename, best_model_name)
    else:
        os.remove(filename)
    print(f'filename {filename} best_model_name{best_model_name} ; best_trial_number {study.best_trial.number}')

In [ ]:
lgbm_study = optuna.create_study(direction='maximize')
lgbm_study.optimize(lgbm_objective, n_trials=100, callbacks=[lgbm_callback])

[I 2021-09-17 06:09:29,251] A new study created in memory with name: no-name-05115bdc-059b-4c5b-b09f-938889ea253a
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:574: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:578: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.



STARTING Params: {'metric': 'AUC', 'verbosity': 0, 'device': 'gpu', 'boosting_type': 'gbdt', 'subsample': 0.5727063392015879, 'learning_rate': 0.010278674171839874, 'num_iterations': 1569, 'subsample_freq': 1, 'lambda_l1': 0.09211554066717548, 'lambda_l2': 0.12079930458940936, 'num_leaves': 162, 'feature_fraction': 0.7936490021224217, 'min_child_samples': 66}


[I 2021-09-17 06:18:36,096] Trial 0 finished with value: 0.8146050196152306 and parameters: {'boosting_type': 'gbdt', 'subsample': 0.5727063392015879, 'eta': 0.010278674171839874, 'num_iterations': 1569, 'lambda_l1': 0.09211554066717548, 'lambda_l2': 0.12079930458940936, 'num_leaves': 162, 'feature_fraction': 0.7936490021224217, 'min_child_samples': 66}. Best is trial 0 with value: 0.8146050196152306.


Trial LGBM AUC 0.8146050196152306 0.5408594042261294 0.9013952571080929
filename ./models/lgbm_0 best_model_name./models/best_lgbm ; best_trial_number 0
STARTING Params: {'metric': 'AUC', 'verbosity': 0, 'device': 'gpu', 'boosting_type': 'gbdt', 'subsample': 0.5723089372660386, 'learning_rate': 0.007122586554732636, 'num_iterations': 12974, 'subsample_freq': 1, 'lambda_l1': 0.5901477129019864, 'lambda_l2': 1.8336514748901417e-05, 'num_leaves': 228, 'feature_fraction': 0.5988502385158633, 'min_child_samples': 25}


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:574: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:578: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.

[I 2021-09-17 06:47:44,328] Trial 1 finished with value: 0.8149008263401365 and parameters: {'boosting_type': 'gbdt', 'subsample': 0.5723089372660386, 'eta': 0.007122586554732636, 'num_iterations': 12974, 'lambda_l1': 0.5901477129019864, 'lambda_l2': 1.8336514748901417e-05, 'num_leaves': 228, 'feature_fraction': 0.5988502385158633, 'min_child_samples': 25}. Best is trial 1 with value: 0.8149008263401365.


...

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:574: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:578: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.

[I 2021-09-18 00:45:51,696] Trial 41 finished with value: 0.8159626259873438 and parameters: {'boosting_type': 'gbdt', 'subsample': 0.573099085855387, 'eta': 0.007244632851899241, 'num_iterations': 37447, 'lambda_l1': 3.5389432163435335, 'lambda_l2': 0.05401143320044411, 'num_leaves': 34, 'feature_fraction': 0.4646475688812316, 'min_child_samples': 40}. Best is trial 39 with value: 0.8161146771285536.


Trial LGBM AUC 0.8159626259873438 0.5410004844425926 0.9025751429349581
filename ./models/lgbm_41 best_model_name./models/best_lgbm ; best_trial_number 39
STARTING Params: {'metric': 'AUC', 'verbosity': 0, 'device': 'gpu', 'boosting_type': 'gbdt', 'subsample': 0.572636020426708, 'learning_rate': 0.0074542698379182265, 'num_iterations': 37562, 'subsample_freq': 1, 'lambda_l1': 0.19494846317268774, 'lambda_l2': 0.06463389018961098, 'num_leaves': 29, 'feature_fraction': 0.4628614480374939, 'min_child_samples': 42}


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:574: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:578: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.

[I 2021-09-18 01:21:52,439] Trial 42 finished with value: 0.8159337125393676 and parameters: {'boosting_type': 'gbdt', 'subsample': 0.572636020426708, 'eta': 0.0074542698379182265, 'num_iterations': 37562, 'lambda_l1': 0.19494846317268774, 'lambda_l2': 0.06463389018961098, 'num_leaves': 29, 'feature_fraction': 0.4628614480374939, 'min_child_samples': 42}. Best is trial 39 with value: 0.8161146771285536.


Trial LGBM AUC 0.8159337125393676 0.5403548117836003 0.9027852386674697
filename ./models/lgbm_42 best_model_name./models/best_lgbm ; best_trial_number 39
STARTING Params: {'metric': 'AUC', 'verbosity': 0, 'device': 'gpu', 'boosting_type': 'gbdt', 'subsample': 0.5724045336241672, 'learning_rate': 0.00751375927856499, 'num_iterations': 37487, 'subsample_freq': 1, 'lambda_l1': 0.1203929410045197, 'lambda_l2': 0.03988094618438434, 'num_leaves': 34, 'feature_fraction': 0.46206876729543683, 'min_child_samples': 61}


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:574: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:578: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.



Trial 39 finished with value: 0.8161146771285536 and parameters: {'boosting_type': 'gbdt', 'subsample': 0.5641972147140464, 'eta': 0.007347787440390321, 'num_iterations': 37743, 'lambda_l1': 0.7809533651443774, 'lambda_l2': 0.04749261940282875, 'num_leaves': 21, 'feature_fraction': 0.4633328194247181, 'min_child_samples': 39}. Best is trial 39 with value: 0.8161146771285536.

Trial LGBM AUC 0.8161146771285536 0.5395740933702102 0.9028642837436492

Kaggle public score:  0.81790

In [ ]:
 # To get the dictionary of parameter name and parameter values:
 # Right now logging is not working as expected so this is pending
#logging.info(f'Return a dictionary of parameter name and parameter values:{lgbm_study.best_params}')

results = []
print(f'Return a dictionary of parameter name and parameter values:{lgbm_study.best_params}')
results.append({'msg':lgbm_study.best_params})
 
# To get the best observed value of the objective function:
#logging.info(f'Return the best observed value of the objective function: {lgbm_study.best_value}')
print(f'Return the best observed value of the objective function: {lgbm_study.best_value}')
results.append({'msg':lgbm_study.best_value})
 
# To get the best trial:
print(f'Return the best trial: {lgbm_study.best_trial}')
results.append({'msg': lgbm_study.best_trial})

pd.DataFrame(results).to_csv('results.csv', index=False)
 
# To get all trials:
#print("Return all the trials:", study.trials)

NameError: ignored

In [ ]:
model_file = open('./models/best_lgbm', 'rb')
colab_lgbm_model_eval = pickle.load(model_file)
model_file.close()

In [ ]:
colab_lgbm_model_eval.fit(X,y)
y_predproba = colab_lgbm_model_eval.predict_proba(X)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:177: UserWarning:

Found `num_iterations` in params. Will use it instead of argument



[LightGBM] [Warning] feature_fraction is set=0.4633328194247181, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4633328194247181
[LightGBM] [Warning] lambda_l2 is set=0.04749261940282875, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.04749261940282875
[LightGBM] [Warning] lambda_l1 is set=0.7809533651443774, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7809533651443774
[LightGBM] [Warning] feature_fraction is set=0.4633328194247181, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4633328194247181
[LightGBM] [Warning] lambda_l2 is set=0.04749261940282875, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.04749261940282875
[LightGBM] [Warning] lambda_l1 is set=0.7809533651443774, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7809533651443774


In [ ]:
roc_auc_score(y, y_predproba[:, 1])

0.9017065674757444

In [ ]:
y_predproba = pd.DataFrame(colab_lgbm_model_eval.predict_proba(test_X))
y_predproba['id'] = y_predproba.index + 957919
y_predproba.to_csv('./artifacts/lightgbm_predproba.csv', index=False)
results_df = y_predproba.rename(columns={1:'claim'}).copy()
results_df = results_df[['id','claim']]
results_df.to_csv('./artifacts/lightgbm_submission.csv', index=False)